In [1]:
from tqdm import tqdm

import FinanceDataReader as fdr
import pandas_datareader as pdr
import yfinance as yf
import numpy as np
import pandas as pd
import re


# Data load

In [2]:
import os
os.getcwd()

'C:\\Users\\PC\\OneDrive\\문서\\GitHub\\Korea_Investment_and_Securities\\scripts\\ipynb'

In [3]:
data_path = "C:\\Users\\PC\\OneDrive\\문서\\GitHub\\Korea_Investment_and_Securities\\datasets\\"

In [7]:
tot_df = pd.read_csv(data_path +"tot_df.csv", encoding = "cp949")
df_theme = pd.read_csv(data_path +'sector.csv', encoding = "cp949")
df_theme = df_theme.dropna()

In [8]:
# tot_df.to_csv("tot_df.csv", index=False)

In [9]:
df_theme.head()

,테마,해당 주식,종목코드
0,빅테크-A(미국),애플,AAPL
1,빅테크-A(미국),알파벳,GOOG
2,빅테크-A(미국),메타,FB
3,빅테크-A(미국),마이크로소프트,MSFT
4,빅테크-A(미국),아마존,AMZN


In [10]:
tot_df.head()

,Symbol,Name,Industry,IndustryCode
0,TSM,Taiwan Semiconductor Manufacturing Co Ltd ADR,반도체 및 반도체 장비,571010.0
1,V,Visa Inc Class A,소프트웨어 및 IT서비스,572010.0
2,JPM,JPMorgan Chase & Co,은행,551010.0
3,UNH,UnitedHealth Group Inc,헬스케어 업체 및 서비스,561020.0
4,JNJ,Johnson & Johnson,제약,562010.0


# Name to Symbol

- 가장 유사한 이름 탐색

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [12]:
etf_name = ["Berkshire Hathaway Inc. Class B",
"JPMorgan Chase & Co.", 
"Citigroup Inc.",
"Goldman Sachs Group, Inc."]

In [13]:
def print_symbol(name, etf_name, tot_df):
    # 탐색 key value 선정
    temp_key_value = name.split(" ")
    key_value = temp_key_value[0]

    # TDM 구축
    total_name = []
    total_name.extend(etf_name)
    total_name.extend(list(tot_df.Name))

    tokenizer = CountVectorizer()
    tdm=tokenizer.fit_transform(total_name)

    first_select = [idx for idx, x in enumerate(total_name) if x.startswith(key_value)]

    if len(first_select) == 2:
        output_idx = first_select[1] - len(etf_name)
    else:
        sims = cosine_similarity(tdm[first_select[0]], tdm[first_select[1:]])
        output_idx = first_select[1:][np.argmax(sims)] - len(etf_name)

    print("name: ", name,"\nresult:", tot_df.Name[output_idx])

    return tot_df.iloc[output_idx].Symbol

In [14]:
symbol = print_symbol(etf_name[3], etf_name, tot_df); symbol

name:  Goldman Sachs Group, Inc. 
result: Goldman Sachs Group Inc


'GS'

* yfinance 기준 주가 데이터 수집

In [15]:
tot_df.head(10)

,Symbol,Name,Industry,IndustryCode
0,TSM,Taiwan Semiconductor Manufacturing Co Ltd ADR,반도체 및 반도체 장비,571010.0
1,V,Visa Inc Class A,소프트웨어 및 IT서비스,572010.0
2,JPM,JPMorgan Chase & Co,은행,551010.0
3,UNH,UnitedHealth Group Inc,헬스케어 업체 및 서비스,561020.0
4,JNJ,Johnson & Johnson,제약,562010.0
5,BRK.B,Berkshire Hathaway Inc Class B,복합 기업,544010.0
6,BAC,Bank of America Corp,은행,551010.0
7,WMT,Walmart Inc,식품 및 약품소매,543010.0
8,PG,Procter & Gamble Co,"개인, 가정 용품 및 서비스",542010.0
9,HD,Home Depot Inc,전문품 상점,534030.0


In [16]:
stock_list = []
col_names = []

In [17]:
start_idx = 0

In [ ]:
for idx, code in tqdm(enumerate(tot_df.Symbol[start_idx:])):
    temp = yf.download(code,start = '2015-01-01', progress = False).loc[:, ['Close']]
    try:
        temp.columns = [code]
    except ValueError:
        start_idx += 1
        continue
        
    stock_list.append(temp)
    start_idx += 1

In [19]:
stock_df = pd.concat(stock_list, axis = 1)
print(stock_df.shape)
stock_df.head()

(1832, 16864)


,TSM,V,JPM,UNH,JNJ,BRK.B,BAC,WMT,PG,HD,...,6628,1432,3039,9388,6695,3456,2452,7680,5073,9276
Date,,,,,,,,,,,,,,,,,,,,,
2014-12-31,22.379999,65.550003,62.580002,101.089996,104.570000,NaN,17.889999,85.879997,91.089996,104.970001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-02,22.280001,66.254997,62.490002,100.779999,104.519997,NaN,17.900000,85.900002,90.440002,103.430000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-05,21.740000,64.792503,60.549999,99.120003,103.790001,NaN,17.379999,85.650002,90.010002,101.260002,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-06,21.360001,64.375000,58.980000,98.919998,103.279999,NaN,16.860001,86.309998,89.599998,100.949997,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-07,21.670000,65.237503,59.070000,99.930000,105.559998,NaN,16.940001,88.599998,90.070000,104.410004,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
stock_df.reset_index(inplace = True)
stock_df.to_csv(data_path + "stock_df.csv", index =False)

In [ ]:
stock_df.loc[:, (stock_df.isnull().sum(axis = 0) != 1832)]

* NaN인 애들 재크롤링

In [44]:
nan_names = stock_df.columns[(stock_df.isnull().sum(axis = 0) == 1832)]
nan_names = [x.split(".")[0] for x in nan_names if type(x) == str]
nan_names = list(pd.Series(nan_names).unique())

In [46]:
stock_list = []
col_names = []

In [47]:
start_idx = 0

In [ ]:
for idx, code in tqdm(enumerate(nan_names[start_idx:])):
    temp = yf.download(code,start = '2015-01-01', progress = False).loc[:, ['Close']]
    try:
        temp.columns = [code]
    except ValueError:
        start_idx += 1
        continue
        
    stock_list.append(temp)
    start_idx += 1

3it [00:00,  3.34it/s]


1 Failed download:
- UN: No data found, symbol may be delisted


5it [00:01,  3.69it/s]


1 Failed download:
- BF: No data found for this date range, symbol may be delisted


6it [00:01,  3.89it/s]

...

7930it [55:17,  3.00it/s]


1 Failed download:
- 8349: No data found, symbol may be delisted

1 Failed download:

7931it [55:17,  3.44it/s]


- 3850: No data found, symbol may be delisted


7932it [55:17,  3.83it/s]

* 종목 뽑아서 처리

In [61]:
stock_df = pd.read_csv(data_path + "stock_df.csv")
final_st_df = pd.read_csv(data_path + "final_stock_df.csv")

In [62]:
stock_df.head()

,Date,TSM,V,JPM,UNH,JNJ,BRK.B,BAC,WMT,PG,...,6628,1432,3039,9388,6695,3456,2452,7680,5073,9276
0,2014-12-31,22.379999,65.550003,62.580002,101.089996,104.570000,NaN,17.889999,85.879997,91.089996,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-01-02,22.280001,66.254997,62.490002,100.779999,104.519997,NaN,17.900000,85.900002,90.440002,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-01-05,21.740000,64.792503,60.549999,99.120003,103.790001,NaN,17.379999,85.650002,90.010002,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-01-06,21.360001,64.375000,58.980000,98.919998,103.279999,NaN,16.860001,86.309998,89.599998,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015-01-07,21.670000,65.237503,59.070000,99.930000,105.559998,NaN,16.940001,88.599998,90.070000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
final_st_df.tail()

,etf_names,stock_names
153604,AILV,"American International Group, Inc."
153605,AILV,"S&P Global, Inc."
153606,AILV,"BlackRock, Inc."
153607,AILV,"Motorola Solutions, Inc."
153608,AILV,U.S. Dollar


In [64]:
nan_cols = list(stock_df.isnull().sum(axis=0) != 1832)

In [65]:
stock_df = stock_df.loc[:, nan_cols]

In [70]:
stock_df.head()

,Date,TSM,V,JPM,UNH,JNJ,BAC,WMT,PG,HD,...,4295,2769,2796,4098,8927,3161,2330,3719,3077,7859
0,2014-12-31,22.379999,65.550003,62.580002,101.089996,104.570000,17.889999,85.879997,91.089996,104.970001,...,NaN,NaN,NaN,2260.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-01-02,22.280001,66.254997,62.490002,100.779999,104.519997,17.900000,85.900002,90.440002,103.430000,...,NaN,NaN,NaN,2260.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-01-05,21.740000,64.792503,60.549999,99.120003,103.790001,17.379999,85.650002,90.010002,101.260002,...,1143.0,1370.0,508.0,2280.0,112.0,606.0,222.000000,119.0,606.0,168.0
3,2015-01-06,21.360001,64.375000,58.980000,98.919998,103.279999,16.860001,86.309998,89.599998,100.949997,...,1148.0,1361.0,504.0,2230.0,110.0,604.0,211.000000,118.0,609.0,161.0
4,2015-01-07,21.670000,65.237503,59.070000,99.930000,105.559998,16.940001,88.599998,90.070000,104.410004,...,1131.0,1351.0,508.0,2400.0,112.0,620.0,34.583302,128.0,598.0,161.0


In [209]:
def print_symbol(name, etf_name, tot_df):
    # 탐색 key value 선정
    temp_key_value = name.split(" ")
    key_value = temp_key_value[0]

    # TDM 구축
    total_name = [name]
    total_name.extend(etf_name)
    total_name.extend(list(tot_df.Name))

    tokenizer = CountVectorizer(max_features = 500000)
    tdm=tokenizer.fit_transform(total_name)

    first_select = [idx for idx, x in enumerate(total_name) if x.startswith(key_value)]
    first_select = np.array(first_select)[np.where(np.array(first_select) > len(etf_name))[0]]
    
    if len(first_select) == 2:
        output_idx = first_select[1] - len(etf_name) -1
        
    elif len(first_select) == 0:
        return "zero"
    
    else:
        sims = cosine_similarity(tdm[0], tdm[first_select])
        output_idx = first_select[np.argmax(sims)] - len(etf_name) -1

    print("name: ", name,"\nresult:", tot_df.Name[output_idx])

    return tot_df.iloc[output_idx].Symbol

In [211]:
symbol = print_symbol(final_st_df.stock_names[10], list(final_st_df.stock_names), tot_df); symbol

'zero'

* 주가 정제

In [213]:
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
from fastdtw import fastdtw

import pickle
import operator

In [218]:
scaler = StandardScaler()

In [221]:
stock_df.iloc[:, 1:] = stock_df.iloc[:, 1:].apply(lambda x: x.fillna(x.mean()),axis=0)

In [222]:
stock_df.head()

,Date,TSM,V,JPM,UNH,JNJ,BAC,WMT,PG,HD,...,4295,2769,2796,4098,8927,3161,2330,3719,3077,7859
0,2014-12-31,22.379999,65.550003,62.580002,101.089996,104.570000,17.889999,85.879997,91.089996,104.970001,...,1289.532026,1320.379747,564.28513,2260.0,165.018963,575.142484,182.612052,115.973451,645.091024,179.515033
1,2015-01-02,22.280001,66.254997,62.490002,100.779999,104.519997,17.900000,85.900002,90.440002,103.430000,...,1289.532026,1320.379747,564.28513,2260.0,165.018963,575.142484,182.612052,115.973451,645.091024,179.515033
2,2015-01-05,21.740000,64.792503,60.549999,99.120003,103.790001,17.379999,85.650002,90.010002,101.260002,...,1143.000000,1370.000000,508.00000,2280.0,112.000000,606.000000,222.000000,119.000000,606.000000,168.000000
3,2015-01-06,21.360001,64.375000,58.980000,98.919998,103.279999,16.860001,86.309998,89.599998,100.949997,...,1148.000000,1361.000000,504.00000,2230.0,110.000000,604.000000,211.000000,118.000000,609.000000,161.000000
4,2015-01-07,21.670000,65.237503,59.070000,99.930000,105.559998,16.940001,88.599998,90.070000,104.410004,...,1131.000000,1351.000000,508.00000,2400.0,112.000000,620.000000,34.583302,128.000000,598.000000,161.000000


In [256]:
tot_df.head()

,Symbol,Name,Industry,IndustryCode
0,TSM,Taiwan Semiconductor Manufacturing Co Ltd ADR,반도체 및 반도체 장비,571010.0
1,V,Visa Inc Class A,소프트웨어 및 IT서비스,572010.0
2,JPM,JPMorgan Chase & Co,은행,551010.0
3,UNH,UnitedHealth Group Inc,헬스케어 업체 및 서비스,561020.0
4,JNJ,Johnson & Johnson,제약,562010.0


In [223]:
sc_df = scaler.fit_transform(stock_df.iloc[:, 1:])

In [225]:
stock_df.iloc[:, 1:] = sc_df

In [226]:
stock_df.head()

,Date,TSM,V,JPM,UNH,JNJ,BAC,WMT,PG,HD,...,4295,2769,2796,4098,8927,3161,2330,3719,3077,7859
0,2014-12-31,-0.916081,-1.316624,-1.278153,-1.394382,-1.335621,-0.987985,-0.503393,-0.486761,-1.278310,...,0.000000,0.000000,0.000000,0.418451,4.660508e-16,-2.092142e-15,4.123102e-16,0.000000,0.000000,0.000000
1,2015-01-02,-0.919113,-1.303987,-1.281059,-1.397518,-1.338015,-0.986837,-0.502670,-0.513431,-1.299160,...,0.000000,0.000000,0.000000,0.418451,4.660508e-16,-2.092142e-15,4.123102e-16,0.000000,0.000000,0.000000
2,2015-01-05,-0.935487,-1.330202,-1.343708,-1.414309,-1.372960,-1.046548,-0.511707,-0.531074,-1.328539,...,-1.622380,0.315288,-2.098144,0.493375,-8.693893e-01,5.678609e-01,5.713961e-01,0.144945,-0.417059,-0.471633
3,2015-01-06,-0.947009,-1.337686,-1.394409,-1.416333,-1.397375,-1.106259,-0.487848,-0.547898,-1.332736,...,-1.567021,0.258102,-2.247252,0.306066,-9.021847e-01,5.310555e-01,4.118204e-01,0.097054,-0.385053,-0.758339
4,2015-01-07,-0.937609,-1.322226,-1.391502,-1.406116,-1.288229,-1.097073,-0.405063,-0.528613,-1.285892,...,-1.755242,0.194562,-2.098144,0.942916,-8.693893e-01,8.254983e-01,-2.147435e+00,0.575964,-0.502411,-0.758339


In [253]:
sector_df.sector.unique().shape

(59,)

In [249]:
sector_df = pd.read_csv(data_path + "sector_df.csv")
sector_df.head()

,sector,etf_names,etf_links
0,Aerospace & Defense,ITA,https://www.etf.com/ITA
1,Aerospace & Defense,XAR,https://www.etf.com/XAR
2,Aerospace & Defense,PPA,https://www.etf.com/PPA
3,Aerospace & Defense,DFEN,https://www.etf.com/DFEN
4,Aerospace & Defense,SHLD,https://www.etf.com/SHLD


In [251]:
sector_df.shape

(1662, 3)

In [250]:
final_st_df.head()

,etf_names,stock_names
0,KORP,U.S. Dollar
1,KORP,Bank of America Corporation 3.419% 20-DEC-2028
2,KORP,Morgan Stanley 2.188% 28-APR-2026
3,KORP,HSBC Holdings Plc 2.099% 04-JUN-2026
4,KORP,Verizon Communications Inc. 4.329% 21-SEP-2028


In [252]:
final_st_df.shape

(153609, 2)

In [254]:
sector_df = pd.merge(sector_df, final_st_df, on = "etf_names", how = "inner")

In [255]:
sector_df.head()

,sector,etf_names,etf_links,stock_names
0,Artificial Intelligence,BOTZ,https://www.etf.com/BOTZ,NVIDIA Corporation
1,Artificial Intelligence,BOTZ,https://www.etf.com/BOTZ,"Intuitive Surgical, Inc."
2,Artificial Intelligence,BOTZ,https://www.etf.com/BOTZ,"Upstart Holdings, Inc."
3,Artificial Intelligence,BOTZ,https://www.etf.com/BOTZ,"Azenta, Inc."
4,Artificial Intelligence,BOTZ,https://www.etf.com/BOTZ,John Bean Technologies Corporation


In [247]:
group_df = sector_df.groupby(["sector", "stock_names"])['etf_names'].count().reset_index()

In [248]:
group_df.head()

,sector,stock_names,etf_names
0,Artificial Intelligence,"2U, Inc.",1
1,Artificial Intelligence,3D Systems Corporation,4
2,Artificial Intelligence,3M Company,1
3,Artificial Intelligence,ABB Ltd.,3
4,Artificial Intelligence,"ABIOMED, Inc.",1


In [ ]:
code_list = df_theme.loc[df_theme.테마 == unique_theme[1]].종목코드.tolist()
code_list = df_theme.종목코드.tolist()

In [ ]:
code_dict = {x:{} for x in df_theme.종목코드.tolist()}

In [ ]:
def ex_nan(inputs) :
  return(np.array(inputs)[[~np.isnan(x) for x in np.array(inputs)]])

In [ ]:
for code in tqdm(code_list):
  try:
    a = theme_in_sc.loc[:, [code]]
  except KeyError:
    continue
  b = theme_in_sc.loc[:, ~theme_in_sc.columns.isin([code])]

  for code_b in b.columns:
    dist, path = fastdtw(ex_nan(a), ex_nan(b.loc[:,code_b]))
    if dist == 0:
      continue
    code_dict[code][code_b] = dist

100%|██████████| 124/124 [02:35<00:00,  1.25s/it]


In [ ]:
'''with open('code_dict.pickle', 'wb') as f:
  pickle.dump(code_dict, f)'''

In [ ]:
with open('code_dict.pickle', 'rb') as f:
  code_dict = pickle.load(f)

In [ ]:
df_theme.head()

,테마,해당 주식,종목코드
0,빅테크-A(미국),애플,AAPL
1,빅테크-A(미국),알파벳,GOOG
2,빅테크-A(미국),메타,FB
3,빅테크-A(미국),마이크로소프트,MSFT
4,빅테크-A(미국),아마존,AMZN


In [ ]:
unique_theme

array(['빅테크-A(미국)', '빅테크-C(중국)', '전기차_2차전지(미국, 중국)', 'NFT_코인(미국, 중국)',
       '메타버스', '우주/항공', '백신/치료', '뷰티/운동', '럭셔리', '여행', '프랜차이즈'],
      dtype=object)

In [ ]:
import re

In [ ]:
for idx_0, th in tqdm(enumerate(unique_theme)):
  th_code_list = df_theme.loc[df_theme.테마 == th, "종목코드"]
  th = re.sub("/","_",th)
  for idx, th_code in enumerate(th_code_list):
    if len(list(code_dict[th_code].keys())) == 0:
      continue
    sorted_items = sorted(code_dict[th_code].items(), key=operator.itemgetter(1))
    names = [x[0] for x in sorted_items]
    sims = [x[1] for x in sorted_items]
    names_list = [tot_df.loc[tot_df.Symbol == x, "Name"].values[0] for x in names]

    if idx == 0:
      sector_dict = {"code":names, "name":names_list, th_code+"dtw":sims}
      sector_df = pd.DataFrame.from_dict(sector_dict)
    else:
      sector_dict1 = {"code":names, "name":names_list, th_code+"dtw":sims}
      sector_df1 = pd.DataFrame.from_dict(sector_dict1)
      sector_df = pd.merge(sector_df, sector_df1, on = ["code", "name"], how = "inner")

  sector_df['mean'] = sector_df.iloc[:, 2:].mean(axis = 1)
  sector_df.to_csv(th + "_sector_df.csv", index = False)
  globals()['sector_' + str(idx_0)] = sector_df

11it [02:36, 14.19s/it]


In [ ]:
sector_6.sort_values("mean")

,code,name,PFEdtw,MRKdtw,mean
19,PAYX,Paychex Inc,31.544509,58.536240,45.040374
7,FFIV,F5 Inc,28.390524,69.738214,49.064369
28,LIN,Linde PLC,33.589007,64.927649,49.258328
25,MORN,Morningstar Inc,33.028737,66.520919,49.774828
27,FAST,Fastenal Co,33.316243,66.869108,50.092675
...,...,...,...,...,...
1177,LCI,Lannett Company Inc,257.581044,212.484793,235.032919
1196,PSFE,Paysafe Ltd,273.333148,197.133123,235.233136
1171,TARO,Taro Pharmaceutical Industries Ltd,255.777154,214.919785,235.348470
1170,ML,MoneyLion Inc Class A,255.457004,221.104151,238.280578


In [ ]:
sector_df.sort_values("mean")

,code,name,MCDdtw,SBUXdtw,DPZdtw,YUMdtw,SHAKdtw,CMGdtw,QSRdtw,mean
234,GFS,Globalfoundries Inc,58.833122,41.058897,46.140646,52.547791,48.569034,31.084007,65.049437,49.040419
371,OLPX,Olaplex Holdings Inc,79.538694,46.967615,42.662903,53.220331,58.417225,47.341577,71.337364,57.069387
286,INFA,Informatica Inc Class A,66.433806,74.193651,59.211372,71.842393,46.877293,50.840673,68.049955,62.492735
320,LTH,Life Time Group Holdings Inc,72.894720,50.519187,78.286218,79.851062,51.759953,46.815155,71.192549,64.474120
380,AKA,AKA Brands Holding Corp,81.832630,51.563419,63.051476,94.836916,51.458338,48.981034,71.771825,66.213662
...,...,...,...,...,...,...,...,...,...,...
1032,DELL,Dell Technologies Inc Class C,240.559185,153.910752,213.468730,180.811225,105.099159,127.438585,72.238178,156.217974
254,RRD,RR Donnelley & Sons Co,62.369440,188.721967,146.025362,123.743995,217.812824,157.529385,219.657424,159.408628
176,DLTR,Dollar Tree Inc,51.860693,207.692703,114.768363,114.470006,242.005175,161.043779,236.925420,161.252306
265,NPTN,NeoPhotonics Corp,64.101265,214.148922,134.125397,113.877140,225.705831,161.205880,223.253765,162.345457


# Using FDR

In [ ]:
theme_list = list(df_theme.종목코드)

In [ ]:
theme_stocks = []
theme_names = []

In [ ]:
np.where(code == np.array(theme_list))

(array([86]),)

In [ ]:
for code in theme_list[86:]:
  try:
    theme_stocks.append(fdr.DataReader(code, '2021-06-01', '2022-01-31')['Close'])
  except KeyError:
    continue
  except ValueError:
    continue

  theme_names.append(code)

In [ ]:
theme_stocks_df = pd.concat(theme_stocks, axis = 1)
theme_stocks_df.columns = theme_names

In [ ]:
theme_stocks_df.head()

,AAPL,GOOG,FB,MSFT,AMZN,NVDA,ADBE,CRM,QCOM,AVGO,INTC,CSCO,NFLX,TSM,TSLA,BABA,JD,DIDI,BIDU,TSLA.1,LIVN,LCID,F,GM,FSR,GP,ALB,002460,NFE,NXPI,FNKO,DLPN,ZKIN,NET,MAT,DKNG,TWTR,EBAY,FB.1,PLBY,...,AMZN.1,TSLA.2,PFE,MRK,EL,PG,JNJ,BBWI,ELF,NKE,UA,LULU,PTON,SHOP,SKX,RACE,HOG,DEO,TSLA.3,CCL,HLT,LVS,MAR,BKNG,V,MAR.1,PYPL.1,ABNB,DIS,EXPE,TRIP,MCD,SBUX,DPZ,YUM,SHAK,CMG,QSR,WING,FAT
0,124.28,2429.81,329.13,247.40,3218.65,162.65,495.91,236.20,133.94,468.31,56.89,52.62,499.08,118.51,623.90,219.48,78.33,NaN,201.82,623.90,81.69,21.56,14.81,59.65,13.40,17.09,171.43,13150.0,44.43,207.91,26.13,9.72,5.15,83.07,21.13,49.57,57.44,61.37,329.13,42.69,...,3218.65,623.90,38.50,71.69,304.08,132.88,165.53,57.21,27.57,134.51,19.63,318.62,108.89,1251.56,47.34,212.70,48.43,193.69,623.90,30.15,126.61,59.20,145.18,2328.28,226.63,145.18,259.27,144.31,178.84,177.31,43.34,233.24,113.34,423.00,119.90,90.92,1361.13,70.47,140.25,10.55
1,125.06,2421.28,329.15,247.30,3233.99,167.78,495.77,234.62,133.82,473.73,57.48,52.96,499.24,118.95,605.12,219.59,76.54,NaN,197.85,605.12,79.24,23.56,14.91,59.65,14.69,17.43,170.36,13250.0,41.92,207.85,26.00,9.11,4.58,83.86,21.54,50.97,57.16,63.41,329.15,43.37,...,3233.99,605.12,38.79,72.34,303.66,133.46,166.20,55.09,27.59,134.17,19.55,320.78,108.55,1229.59,46.42,214.41,48.62,194.52,605.12,31.31,125.17,58.92,142.63,2317.90,229.66,142.63,262.17,151.00,177.00,175.17,42.44,233.78,113.00,426.20,119.24,94.81,1362.39,69.73,142.23,10.89
2,123.54,2404.61,326.04,245.71,3187.01,169.70,493.14,230.84,131.78,464.80,56.24,53.33,489.43,116.82,572.84,217.04,75.52,NaN,189.97,572.84,79.26,22.96,15.99,63.46,15.91,17.04,168.17,13250.0,41.03,200.75,23.50,8.90,4.50,81.92,20.86,49.71,57.01,64.07,326.04,44.69,...,3187.01,572.84,38.97,73.91,302.88,135.85,166.04,53.40,27.33,134.17,19.06,317.36,103.10,1199.00,46.63,215.43,48.98,192.88,572.84,30.67,123.13,56.76,140.40,2295.80,228.11,140.40,257.79,144.19,176.24,170.75,42.26,232.45,111.12,425.12,118.72,90.59,1333.04,69.12,137.55,10.78
3,125.89,2451.76,330.35,250.79,3206.22,175.78,504.50,237.48,134.34,475.00,57.37,54.07,494.74,119.51,599.05,219.02,75.43,NaN,192.99,599.05,80.08,23.95,15.97,63.37,15.92,17.12,173.54,13300.0,39.63,202.00,23.54,8.51,4.77,83.08,20.68,50.33,59.00,65.23,330.35,45.92,...,3206.22,599.05,39.15,74.11,303.35,135.95,165.97,53.11,27.30,133.74,19.02,329.52,103.01,1205.74,46.39,216.88,48.05,192.62,599.05,30.54,124.75,56.52,142.06,2315.69,230.14,142.06,263.04,150.73,177.18,171.42,41.26,233.38,111.99,431.14,118.77,89.28,1326.34,69.18,137.80,11.34
4,125.90,2466.09,336.58,253.81,3198.01,176.19,509.47,238.14,133.32,463.78,57.09,53.92,494.66,118.22,605.13,216.90,74.68,NaN,188.92,605.13,80.09,26.50,15.88,63.23,17.40,17.23,170.78,13450.0,40.85,201.15,23.72,8.75,5.02,85.84,20.69,53.51,59.63,65.45,336.58,52.09,...,3198.01,605.13,38.99,72.80,302.96,136.42,164.84,53.45,27.52,133.95,19.25,331.61,109.79,1230.55,46.71,219.37,47.17,193.16,605.13,30.89,124.90,56.52,142.48,2298.50,231.32,142.48,260.60,148.97,176.99,170.35,42.25,231.69,111.33,437.92,118.32,95.11,1323.26,68.67,139.91,11.60


In [ ]:
# theme_stocks_df.to_csv("theme_stocks.csv", index =False)

In [ ]:
theme_stocks_df = pd.read_csv("theme_stocks.csv")

In [ ]:
df_theme.head()

,테마,해당 주식,종목코드
0,빅테크-A(미국),애플,AAPL
1,빅테크-A(미국),알파벳,GOOG
2,빅테크-A(미국),메타,FB
3,빅테크-A(미국),마이크로소프트,MSFT
4,빅테크-A(미국),아마존,AMZN


In [ ]:
tot_df.head()

,Symbol,Name,Industry,IndustryCode
0,TSM,Taiwan Semiconductor Manufacturing Co Ltd ADR,반도체 및 반도체 장비,571010
1,V,Visa Inc Class A,소프트웨어 및 IT서비스,572010
2,JPM,JPMorgan Chase & Co,은행,551010
3,UNH,UnitedHealth Group Inc,헬스케어 업체 및 서비스,561020
4,JNJ,Johnson & Johnson,제약,562010


In [ ]:
industry_list = tot_df.loc[tot_df.Symbol.isin(df_theme.종목코드),:].Industry.unique()

In [ ]:
tot_df_in = tot_df.loc[tot_df.Industry.isin(industry_list)]; tot_df_in.shape

(8065, 4)

In [ ]:
theme_in = []
theme_names = []

In [ ]:
for code in tqdm(tot_df_in.Symbol):
  try:
    theme_in.append(fdr.DataReader(code, '2021-06-01', '2022-01-31')['Close'])
  except KeyError:
    continue
  except ValueError:
    continue

  theme_names.append(code)

100%|██████████| 8065/8065 [1:10:11<00:00,  1.91it/s]


In [ ]:
theme_in_df = pd.concat(theme_in, axis = 1)
theme_in_df.columns = theme_names

In [ ]:
# theme_in_df.to_csv("theme_in_df.csv", index = False)

In [ ]:
theme_in_df = pd.read_csv("theme_in_df.csv")

In [ ]:
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
from fastdtw import fastdtw

import pickle
import operator

In [ ]:
scaler = StandardScaler()

In [ ]:
theme_in_sc = scaler.fit_transform(theme_in_df)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/extmath.py:985: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.7/dist-packages/sklearn/utils/extmath.py:990: RuntimeWarning: invalid value encountered in true_divide
  T = new_sum / new_sample_count
/usr/local/lib/python3.7/dist-packages/sklearn/utils/extmath.py:1020: RuntimeWarning: invalid value encountered in true_divide
  new_unnormalized_variance -= correction ** 2 / new_sample_count


In [ ]:
theme_in_sc = pd.DataFrame(theme_in_sc, columns = theme_in_df.columns)

In [ ]:
theme_in_sc.shape

(175, 1217)

In [ ]:
theme_in_sc.head()

,TSM,V,JNJ,PG,HD,MA,BABA,PFE,DIS,TM,KO,ABBV,LLY,TMO,ABT,NKE,ORCL,ACN,CRM,DHR,MRK,NVS,MCD,NVO,LOW,LIN,BMY,RTX,SAP,MDT,UL,SONY,BA,DE,IBM,DEO,NOW,GSK,CAT,LMT,...,002025,002690,000547,000400,000860,000997,002670,000990,002810,002250,002960,000970,000810,002390,002530,000030,000650,300080,000420,000950,002210,002540,002020,002990,002550,000910,002880,000680,002170,002300,000070,000815,000850,000700,300300,002355,002365,002630,002620,000687
0,-0.011203,0.164016,-0.312882,-1.461858,-1.034969,0.133729,1.565127,-1.349979,0.821313,-1.151640,-0.429394,-0.668152,-2.811619,-2.219179,-2.329895,-1.990233,-1.699424,-1.775676,-0.985123,-2.319409,-1.659477,0.168633,-1.137565,-2.024923,-1.070353,-0.602096,0.535698,0.780628,-0.239532,0.403656,1.616226,-1.172258,2.322877,0.651766,0.955565,-0.729743,-2.240246,-1.384802,3.132330,1.255687,...,0.853093,1.311524,2.036518,1.294404,0.583670,NaN,NaN,-1.402684,0.995194,NaN,-0.319417,0.498066,-0.787342,1.652292,NaN,NaN,2.039887,-0.469143,NaN,-0.146460,1.542639,NaN,-0.684561,-0.852512,NaN,1.153135,-1.195939,0.269075,1.653742,NaN,0.957888,-0.669667,0.746944,3.233125,NaN,0.137668,NaN,2.188377,1.663012,NaN
1,0.072681,0.394277,-0.179134,-1.394935,-1.125536,0.567692,1.568350,-1.302943,0.679482,-0.619130,-0.324328,-0.759770,-2.769999,-2.180470,-2.187783,-2.018070,-1.572608,-1.747557,-1.052237,-2.455833,-1.483276,0.339327,-1.087849,-2.058789,-1.115248,-0.741510,0.434980,0.645140,-0.396727,0.301679,1.765998,-1.285048,2.382513,0.090752,1.123526,-0.628686,-2.275731,-1.365273,3.196942,1.436727,...,0.853093,1.373466,1.832576,1.261502,0.682896,NaN,NaN,-1.195347,1.104463,NaN,-0.182054,0.619772,-0.558512,1.652292,NaN,NaN,2.396694,-0.551452,NaN,-0.034092,1.491726,NaN,-0.605762,-0.125581,NaN,1.096909,-1.099034,0.284920,1.744329,NaN,1.015675,-0.741912,1.070749,2.987142,NaN,0.174930,NaN,2.099248,1.677423,NaN
2,-0.333394,0.276487,-0.211074,-1.119164,-1.178464,0.258509,1.493632,-1.273747,0.620900,-0.241780,-0.257468,-0.668152,-2.560726,-2.248739,-1.893317,-2.018070,-1.424051,-1.813880,-1.212801,-2.498465,-1.057683,0.432432,-1.210296,-1.976115,-1.134786,-0.832890,0.462002,0.663958,-0.626337,0.226483,1.792624,-1.215561,2.027378,0.246984,1.104864,-0.828366,-2.404038,-1.209049,3.232017,1.565798,...,0.896594,1.212753,2.189474,1.919540,0.749047,NaN,NaN,-0.853850,1.104463,NaN,0.367399,0.480680,-0.787342,1.652292,NaN,NaN,2.329793,-0.539694,NaN,0.265558,1.237160,NaN,-0.511203,0.056152,NaN,1.420208,-1.074807,0.324532,1.744329,NaN,1.015675,-0.886400,1.145473,2.790356,NaN,0.305347,NaN,2.158667,1.648600,NaN
3,0.179442,0.430753,-0.225047,-1.107625,-1.190814,0.516674,1.551648,-1.244552,0.693357,0.098332,0.029076,-0.651186,-2.600587,-2.188388,-1.881795,-2.053277,-1.097952,-1.705685,-0.930752,-2.444748,-1.003468,0.541056,-1.124675,-1.849613,-1.150998,-0.742681,0.496394,0.791919,-0.266026,0.306829,1.825907,-1.020192,2.000576,0.085781,1.310150,-0.860023,-2.380216,-1.116290,3.248632,1.951295,...,0.418081,1.296457,1.679620,1.886638,0.633283,NaN,NaN,-0.744084,0.995194,NaN,0.642125,0.411134,-0.558512,1.546115,NaN,NaN,1.906085,-0.616124,NaN,0.377926,1.186247,NaN,-0.621522,0.237885,NaN,1.448320,-0.953676,0.387910,1.537272,NaN,0.900100,-0.814156,0.996025,3.019940,NaN,0.379871,NaN,2.336925,1.677423,NaN
4,-0.066490,0.520426,-0.450623,-1.053394,-1.221689,0.397426,1.489530,-1.270503,0.678712,-0.101515,-0.066439,-0.577665,-1.398879,-2.264223,-1.790894,-2.036083,-0.907728,-1.735638,-0.902717,-2.531293,-1.358580,0.512237,-1.280266,-1.824711,-1.136864,-1.227701,0.454632,0.773101,-0.084104,0.318160,1.845876,-1.074573,2.184173,-0.000146,1.376017,-0.794274,-2.399766,-1.189520,2.855422,1.803378,...,0.418081,1.242887,1.220751,1.886638,0.649821,NaN,NaN,-0.866047,0.885926,NaN,0.962639,0.532839,-0.329681,1.546115,NaN,NaN,1.526978,-0.692554,NaN,0.040820,1.186247,NaN,-0.668801,0.722506,NaN,1.588885,-0.953676,0.292842,1.589036,NaN,0.842313,-0.669667,0.821668,2.872350,NaN,0.603442,NaN,2.069538,1.691835,NaN


In [ ]:
df_theme.head()

,테마,해당 주식,종목코드
0,빅테크-A(미국),애플,AAPL
1,빅테크-A(미국),알파벳,GOOG
2,빅테크-A(미국),메타,FB
3,빅테크-A(미국),마이크로소프트,MSFT
4,빅테크-A(미국),아마존,AMZN


In [ ]:
unique_theme = df_theme.테마.unique()
unique_theme

array(['빅테크-A(미국)', '빅테크-C(중국)', '전기차_2차전지(미국, 중국)', 'NFT_코인(미국, 중국)',
       '메타버스', '우주/항공', '백신/치료', '뷰티/운동', '럭셔리', '여행', '프랜차이즈'],
      dtype=object)

In [ ]:
code_list = df_theme.loc[df_theme.테마 == unique_theme[1]].종목코드.tolist()
code_list = df_theme.종목코드.tolist()

In [ ]:
code_dict = {x:{} for x in df_theme.종목코드.tolist()}

In [ ]:
def ex_nan(inputs) :
  return(np.array(inputs)[[~np.isnan(x) for x in np.array(inputs)]])

In [ ]:
for code in tqdm(code_list):
  try:
    a = theme_in_sc.loc[:, [code]]
  except KeyError:
    continue
  b = theme_in_sc.loc[:, ~theme_in_sc.columns.isin([code])]

  for code_b in b.columns:
    dist, path = fastdtw(ex_nan(a), ex_nan(b.loc[:,code_b]))
    if dist == 0:
      continue
    code_dict[code][code_b] = dist

100%|██████████| 124/124 [02:35<00:00,  1.25s/it]


In [ ]:
'''with open('code_dict.pickle', 'wb') as f:
  pickle.dump(code_dict, f)'''

In [ ]:
with open('code_dict.pickle', 'rb') as f:
  code_dict = pickle.load(f)

In [ ]:
df_theme.head()

,테마,해당 주식,종목코드
0,빅테크-A(미국),애플,AAPL
1,빅테크-A(미국),알파벳,GOOG
2,빅테크-A(미국),메타,FB
3,빅테크-A(미국),마이크로소프트,MSFT
4,빅테크-A(미국),아마존,AMZN


In [ ]:
unique_theme

array(['빅테크-A(미국)', '빅테크-C(중국)', '전기차_2차전지(미국, 중국)', 'NFT_코인(미국, 중국)',
       '메타버스', '우주/항공', '백신/치료', '뷰티/운동', '럭셔리', '여행', '프랜차이즈'],
      dtype=object)

In [ ]:
import re

In [ ]:
for idx_0, th in tqdm(enumerate(unique_theme)):
  th_code_list = df_theme.loc[df_theme.테마 == th, "종목코드"]
  th = re.sub("/","_",th)
  for idx, th_code in enumerate(th_code_list):
    if len(list(code_dict[th_code].keys())) == 0:
      continue
    sorted_items = sorted(code_dict[th_code].items(), key=operator.itemgetter(1))
    names = [x[0] for x in sorted_items]
    sims = [x[1] for x in sorted_items]
    names_list = [tot_df.loc[tot_df.Symbol == x, "Name"].values[0] for x in names]

    if idx == 0:
      sector_dict = {"code":names, "name":names_list, th_code+"dtw":sims}
      sector_df = pd.DataFrame.from_dict(sector_dict)
    else:
      sector_dict1 = {"code":names, "name":names_list, th_code+"dtw":sims}
      sector_df1 = pd.DataFrame.from_dict(sector_dict1)
      sector_df = pd.merge(sector_df, sector_df1, on = ["code", "name"], how = "inner")

  sector_df['mean'] = sector_df.iloc[:, 2:].mean(axis = 1)
  sector_df.to_csv(th + "_sector_df.csv", index = False)
  globals()['sector_' + str(idx_0)] = sector_df

11it [02:36, 14.19s/it]


In [ ]:
sector_6.sort_values("mean")

,code,name,PFEdtw,MRKdtw,mean
19,PAYX,Paychex Inc,31.544509,58.536240,45.040374
7,FFIV,F5 Inc,28.390524,69.738214,49.064369
28,LIN,Linde PLC,33.589007,64.927649,49.258328
25,MORN,Morningstar Inc,33.028737,66.520919,49.774828
27,FAST,Fastenal Co,33.316243,66.869108,50.092675
...,...,...,...,...,...
1177,LCI,Lannett Company Inc,257.581044,212.484793,235.032919
1196,PSFE,Paysafe Ltd,273.333148,197.133123,235.233136
1171,TARO,Taro Pharmaceutical Industries Ltd,255.777154,214.919785,235.348470
1170,ML,MoneyLion Inc Class A,255.457004,221.104151,238.280578


In [ ]:
sector_df.sort_values("mean")

,code,name,MCDdtw,SBUXdtw,DPZdtw,YUMdtw,SHAKdtw,CMGdtw,QSRdtw,mean
234,GFS,Globalfoundries Inc,58.833122,41.058897,46.140646,52.547791,48.569034,31.084007,65.049437,49.040419
371,OLPX,Olaplex Holdings Inc,79.538694,46.967615,42.662903,53.220331,58.417225,47.341577,71.337364,57.069387
286,INFA,Informatica Inc Class A,66.433806,74.193651,59.211372,71.842393,46.877293,50.840673,68.049955,62.492735
320,LTH,Life Time Group Holdings Inc,72.894720,50.519187,78.286218,79.851062,51.759953,46.815155,71.192549,64.474120
380,AKA,AKA Brands Holding Corp,81.832630,51.563419,63.051476,94.836916,51.458338,48.981034,71.771825,66.213662
...,...,...,...,...,...,...,...,...,...,...
1032,DELL,Dell Technologies Inc Class C,240.559185,153.910752,213.468730,180.811225,105.099159,127.438585,72.238178,156.217974
254,RRD,RR Donnelley & Sons Co,62.369440,188.721967,146.025362,123.743995,217.812824,157.529385,219.657424,159.408628
176,DLTR,Dollar Tree Inc,51.860693,207.692703,114.768363,114.470006,242.005175,161.043779,236.925420,161.252306
265,NPTN,NeoPhotonics Corp,64.101265,214.148922,134.125397,113.877140,225.705831,161.205880,223.253765,162.345457
